# Acquire data

In [ ]:
import pandas as pd

path = './data/2023.xls'

df = pd.read_excel(path, sheet_name='1月')
col = list(df.columns)

events = []
for i in range(len(col)):
    events.append(df['使用过程'][i])
    print(df['使用过程'][i])

# Pipeline

In [ ]:
from transformers import pipeline

text = '接诊医生为患者测量空腹血糖，血糖条无法自动吸血，导致无法测量，更换新的血糖条后，顺利为患者测量了空腹血糖。'
summarizer = pipeline("summarization", model="./hub/mt5-small-finetuned-on-mT5-lcsts")
print(summarizer(text, max_length=30, min_length=10, do_sample=False))

In [2]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

os.environ['TRANSFORMERS_OFFLINE'] = '1'
source_model = "./hub/mt5-small-finetuned-on-mT5-lcsts"

tokenizer = AutoTokenizer.from_pretrained("./hub/mt5-small-finetuned-on-mT5-lcsts", cache_dir = './hub/mt5-small-finetuned-on-mT5-lcsts')
model = AutoModelForSeq2SeqLM.from_pretrained("./hub/mt5-small-finetuned-on-mT5-lcsts", cache_dir = '.hub/mt5-small-finetuned-on-mT5-lcsts')

/home/alba/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
text = '接诊医生为患者测量空腹血糖，血糖条无法自动吸血，导致无法测量，更换新的血糖条后，顺利为患者测量了空腹血糖。'
label = '医生为患者测量空腹血糖'

from torch.utils.data import DataLoader

input_ids = tokenizer.encode(text, return_tensors='pt')
label_ids = tokenizer.encode(label, return_tensors='pt')
out = model(input_ids, label_ids)
out.loss

RuntimeError: The size of tensor a (42) must match the size of tensor b (10) at non-singleton dimension 3

In [ ]:
for text in events:
    if len(text) <= 25:
        continue
    print('input:{}'.format(text))
    input_ids = tokenizer.encode(text, return_tensors='pt')
    summary_ids = model.generate(input_ids,
                                min_length=5,
                                max_length=50,
                                num_beams=10,
                                repetition_penalty=2.5,
                                length_penalty=1.0,
                                early_stopping=True,
                                no_repeat_ngram_size=2,
                                use_cache=True,
                                do_sample = True,
                                temperature = 0.8,
                                top_k = 50,
                                top_p = 0.95)
    
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print('output:{}\n'.format(summary_text))